In [15]:
import pandas as pd # dataframes
import numpy as np # algebra & calculus
import nltk # text preprocessing & manipulation
import matplotlib.pyplot as plt # plotting
import seaborn as sns # plotting

import gc
color = sns.color_palette() # adjusting plotting style
import warnings
warnings.filterwarnings('ignore') # silence annoying warnings

# read aggregated file, 
# preassign dtypes to reduce memory usage
dtype = {'add_to_cart_order': np.int16,
         'aisle_id': np.int32,
         'day_since_prior_order': np.float16,
         'department_id': np.int8,
         'eval_set': np.int8,
         'order_dow': np.int8,
         'order_hour_of_day': np.int8,
         'order_id': np.int32,
         'order_number': np.int32,
         'product_id': np.int32,
         'reordered': np.int8,
         'user_id': np.int32      
         }
order_detailed = pd.read_csv('data/agg.csv', engine='c', dtype=dtype)

dtype = {'order_id': np.int32,
         'user_id': np.int32,
         'eval_set': np.int8,
         'order_number': np.int32,
         'order_dow': np.int8,
         'order_hour': np.int8,
         'days_since_prior_order': np.float16
        }
# test_detailed = pd.read_csv('data/tset.csv', engine='c', dtype=dtype)

In [16]:
test = pd.read_csv('data/sample_submission.csv', engine='c')
test.head()

,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259


In [17]:
# split train and test
order_detailed_train = order_detailed.loc[~order_detailed['order_id'].isin(test.order_id)]
# order_detailed_test = order_detailed.loc[order_detailed['order_id'].isin(test.order_id)]

In [24]:
grouped_df = order_detailed_train.groupby('user_id')
# grouped_df = order_detailed.groupby('user_id')
# reduce memory
del order_detailed, order_detailed_train
gc.collect()

460

In [40]:
grouped_df.get_group(1).sort_values(['order_number', 'add_to_cart_order'])

,add_to_cart_order,aisle_id,days_since_prior_order,department_id,eval_set,order_dow,order_hour_of_day,order_id,order_number,product_id,reordered,user_id
25461281,1,77,NaN,7,0,2,8,2539329,1,196,0,1
25461282,2,91,NaN,16,0,2,8,2539329,1,14084,0,1
25461283,3,23,NaN,19,0,2,8,2539329,1,12427,0,1
25461284,4,23,NaN,19,0,2,8,2539329,1,26088,0,1
25461285,5,54,NaN,17,0,2,8,2539329,1,26405,0,1
24127361,1,77,15.0,7,0,3,7,2398795,2,196,1,1
24127362,2,117,15.0,19,0,3,7,2398795,2,10258,0,1
24127363,3,23,15.0,19,0,3,7,2398795,2,12427,1,1
24127364,4,24,15.0,4,0,3,7,2398795,2,13176,0,1
24127365,5,23,15.0,19,0,3,7,2398795,2,26088,1,1


In [74]:
# pred by calculated each user
import random
users = random.sample(grouped_df.groups.keys(), 1)
users = [1]
for user in users:
    user_df = grouped_df.get_group(user)
    grouped_product = user_df[user_df.eval_set == 0].groupby('product_id')
    products = grouped_product.groups.keys()
    y_real = user_df[user_df.eval_set == 1]['product_id'].values.tolist()
    for product in [196]:
        product_df = grouped_product.get_group(product)

In [75]:
product_df

,add_to_cart_order,aisle_id,days_since_prior_order,department_id,eval_set,order_dow,order_hour_of_day,order_id,order_number,product_id,reordered,user_id
5474015,1,77,28.0,7,0,4,15,431534,5,196,1,1
5872712,1,77,21.0,7,0,3,12,473747,3,196,1,1
6597544,1,77,20.0,7,0,1,9,550135,7,196,1,1
22760691,1,77,29.0,7,0,4,7,2254736,4,196,1,1
23145063,4,77,0.0,7,0,1,16,2295261,9,196,1,1
24127361,1,77,15.0,7,0,3,7,2398795,2,196,1,1
25461281,1,77,NaN,7,0,2,8,2539329,1,196,0,1
25565883,1,77,30.0,7,0,4,8,2550362,10,196,1,1
30859423,2,77,14.0,7,0,1,14,3108588,8,196,1,1
33311687,1,77,19.0,7,0,2,7,3367565,6,196,1,1


In [72]:
user_df[user_df.eval_set == 1]['product_id'].values.tolist()

[196, 25133, 38928, 26405, 39657, 10258, 13032, 26088, 27845, 49235, 46149]